In [1]:
import numpy as np
from qiskit.quantum_info.operators.predicates import (is_hermitian_matrix,
                                                      is_unitary_matrix)
from scipy.linalg import expm
import tensornetwork as tn

In [82]:
n_weight = 16
gates = 6

weights = np.random.normal(size=(gates, n_weight))


def unitary_from_hermitian(hermitian):
    """Generates a unitary matrix from a hermitian matrix.
        The formula is U = e^(i*H).

    Args:
        hermitian: A hermitian matrix.

    Returns:
        unitary: The resulting unitarian matrix.

    Raises:
        AssertionError: If the resulting matrix is not unitarian.
    """
    unitary = np.array(expm(1j * hermitian))
    assert is_unitary_matrix(unitary)
    return unitary


def hermitian_from_weights(weights, dimension):
    """Generates a  complex hermitian matrix from a set of weights.
        The hermitian is constructed by an upper triangle matrix which then is
        added to its transpose. The first dimension weights are used for the real
        diagonal values, the next values are used for the real parts of the upper
        triangle the rest for the imaginarie parts.

    Args:
        weights: List of weights.
        dimension: size of the matrix.

    Returns:
        hermitian: The resulting hermitian matrix.

    Raises:
        AssertionError: If the resulting matrix is not hermitian.
    """
    diagonals = weights[:dimension]
    dim = ((dimension**2 - dimension) // 2) + dimension
    reals = weights[dimension:dim]
    imaginaries = weights[dim:]
    assert reals.shape == imaginaries.shape
    diag = np.matrix(np.diag(diagonals))
    hermitian = np.matrix(np.zeros((dimension, dimension), dtype=complex))

    hermitian[np.triu_indices(dimension, 1)] = np.array(
        [complex(a, b) for a, b in zip(reals, imaginaries)])
    hermitian = hermitian + hermitian.H + diag  # tril and triu don't use the same ordering!
    assert is_hermitian_matrix(hermitian)
    return hermitian


def unitaries_from_weights(weights, dimension=4):
    """Wrapper function to generate unitary matricies from weight list.

    Args:
        weights: The weights to transform.

    Returns:
        unitaries: The resulting list of unitary matrices.
    """
    unitaries = []
    for weight in weights:
        unitaries.append(
            unitary_from_hermitian(hermitian_from_weights(weight, dimension)))
    return unitaries



In [83]:
unitaries = unitaries_from_weights(weights)

In [20]:
qubits = 16
v = 2

dimension = 4**v
steps = int(np.log2(qubits))-1
gates_per_step = []
for i in range(1, steps + 1):
    gates_per_step.append((qubits // 2) //(2 ** i))

In [85]:
weights = np.random.normal(size=(int(sum(gates_per_step)), 2**(4*v)))

In [87]:
unitaries = unitaries_from_weights(weights, dimension)

In [93]:
# Generating an artificial image

dim = 4
data = np.random.normal(size=(dim**2))

In [104]:
# Preparing the unitary matrixes in the tensor network
tensor_network = []

redistribute_indexes = [2 for i in range(int(np.log2(unitaries[0].size)))]

for unitary in unitaries:
    tensor_network.append(tn.Node(unitary.reshape(avg)))
    tensor_network.append(tn.Node(unitary.conjugate().T.reshape(avg)))

In [105]:
# Preparing the tensors holding the data

feature_map = np.zeros((data.shape[0], 2))
tensor_data = []

for i in range(dim):
    for k in range(dim):
        idx= i*dim+k
        feature_map[idx][0] = np.cos(np.pi / 2 * data[idx])
        feature_map[idx][1] = np.sin(np.pi / 2 * data[idx])
        tensor_data.append(tn.Node(feature_map[idx][:,np.newaxis]))
        tensor_network[idx][k]^tensor_data[idx][0]
    for k in range(dim):
        tensor_data.append(tn.Node(feature_map[i*dim+k][:,np.newaxis].T))

In [106]:
tensor_data[0][0]^tensor_network[0][0]


Edge('__unnamed_node__'[0] -> '__unnamed_node__'[0] )

In [75]:
# Connect the dangling edges
for i in range(dim**2):
    tensor_network[i//dim][i]^tensor_data[2*i][0]


14

[2, 2, 2, 2, 2, 2, 2, 2]

In [37]:
U.size

256